In [1]:
def coding(text):
  import numpy as np
  import zlib
  crc32=zlib.crc32(text.encode('utf-8'))
  a=[]
  for elem in text:
    a.append(ord(elem))
  x=np.zeros((len(a)//3+min(len(a)%3,1),124))
  for index in range(0,len(a),3):
    b=np.zeros(20)
    for i in range(20):
      if 2**(19-i)<=a[index]:
        b[i]=1
        a[index]-=2**(19-i)
    k=0
    for i in range(25):
     if 2**k==i+1:
      k+=1
     else:
      x[index//3][i]=b[i-k]
    b=np.zeros(20)
    if index+1<len(a):
      for i in range(20):
        if 2**(19-i)<=a[index+1]:
          b[i]=1
          a[index+1]-=2**(19-i)
    for i in range(20):
      if i<6:
       x[index//3][i+25]=b[i]
      else:
       x[index//3][i+26]=b[i]      
    b=np.zeros(20)
    if index+2<len(a):
      for i in range(20):
        if 2**(19-i)<=a[index+2]:
          b[i]=1
          a[index+2]-=2**(19-i)      
    for i in range(20):
      if i<17:
       x[index//3][i+46]=b[i]
      else:
       x[index//3][i+47]=b[i]   
  p=[]      
  for ind in range(x.shape[0]):
    y=np.zeros(7)    
    for i in range(7):
      s=0
      for j in range(2**i-1, 124 , 2**(i+1)):
        for k in range(2**i):
          if j+k<124:
            s+=x[ind][j+k]
      y[i]=int(s)%2
    for i in range(7):        
      x[ind][2**i-1]=y[i]
    p.append(y)
  cons=zlib.crc32(str(p).encode('utf-8'))
  return x, p, crc32, cons

In [2]:
def encoding(mes, matr, c32, cs):
  
  import numpy as np
  import zlib
   
  message, mat=mes.copy(), matr.copy()
  if cs==zlib.crc32(str(mat).encode('utf-8')):
    print('контрольная сумма корректна') 
  def con_sum(message):
    p=[]    
    for ind in range(message.shape[0]):
      y=np.zeros(7)  
      for i in range(7):
        s=0
        for j in range(2**i-1, 124, 2**(i+1)):
          for k in range(2**i):
            if j+k<124:
              s+=message[ind][j+k]
        y[i]=int(s)%2
      p.append(y)
    return(p)
  def go_back(x):
    k=0
    w1=[]
    w2=[]
    w3=[]
    for i in range(25):
      if 2**k==i+1:
        k+=1
      else:
        w1.append(x[i])
    for i in range(25,46):
      if 2**k==i+1:
        k+=1
      else:
        w2.append(x[i])
    for i in range(46,124):
      if 2**k==i+1:
        k+=1
      else:
        w3.append(x[i])    
    l1=0    
    for i in range(len(w1)):
      l1+=2**(19-i)*w1[i]
    l=chr(int(l1))
    l2=0    
    for i in range(len(w2)):
      l2+=2**(19-i)*w2[i]
    l3=0    
    for i in range(len(w3)):
      l3+=2**(19-i)*w3[i]    
    if l2!=0:  
      l+=chr(int(l2))
    if l3!=0:
      l+=chr(int(l3))
    return l
  def eq(m1,m2):
    flag=True
    for i in range(len(m1)):
      for j in range (len(m1[0])):
        if m1[i][j]!=m2[i][j]:
          flag=False
    return flag
  for ind in range(message.shape[0]):
    for i in range(7):
      message[ind][2**i-1]=0
  p=con_sum(message)
  if (eq(p,mat)):
    text=''
    for ind in range(message.shape[0]):
      text+=go_back(message[ind])
  else: 
    for ind in range(len(p)):
        err=0  
        for i in range(len(p[ind])):
          if p[ind][i]!=mat[ind][i] and err+2**i<=124:
            err+=2**i
        if err>0:
          message[ind][err-1]=1-message[ind][err-1]
    p=con_sum(message)
    text=''
    for ind in range(message.shape[0]):
      text+=go_back(message[ind])
  if c32==zlib.crc32(text.encode('utf-8')):
    print('сообщение доставлено верно')
  else:
    print('в доставленном сообщении имеются ошибки')
  return text

In [3]:
t="""Язык возник в начале 1980-х годов, когда сотрудник фирмы Bell Labs Бьёрн Страуструп придумал ряд усовершенствований к языку C под собственные нужды[7]. Когда в конце 1970-х годов Страуструп начал работать в Bell Labs над задачами теории очередей (в приложении к моделированию телефонных вызовов), он обнаружил, что попытки применения существующих в то время языков моделирования оказываются неэффективными, а применение высокоэффективных машинных языков слишком сложно из-за их ограниченной выразительности. Так, язык Симула имеет такие возможности, которые были бы очень полезны для разработки большого программного обеспечения, но работает слишком медленно, а язык BCPL достаточно быстр, но слишком близок к языкам низкого уровня и не подходит для разработки большого программного обеспечения.

Вспомнив опыт своей диссертации, Страуструп решил дополнить язык C (преемник BCPL) возможностями, имеющимися в языке Симула. Язык C, будучи базовым языком системы UNIX, на которой работали компьютеры Bell, является быстрым, многофункциональным и переносимым. Страуструп добавил к нему возможность работы с классами и объектами. В результате практические задачи моделирования оказались доступными для решения как с точки зрения времени разработки (благодаря использованию Симула-подобных классов), так и с точки зрения времени вычислений (благодаря быстродействию C). В первую очередь в C были добавлены классы (с инкапсуляцией), наследование классов, строгая проверка типов, inline-функции и аргументы по умолчанию. Ранние версии языка, первоначально именовавшегося «C with classes» («Си с классами»), стали доступны с 1980 года.

Разрабатывая C с классами, Страуструп написал программу cfront — транслятор, перерабатывающий исходный код C с классами в исходный код простого C. Это позволило работать над новым языком и использовать его на практике, применяя уже имеющуюся в UNIX инфраструктуру для разработки на C. Новый язык, неожиданно для автора, приобрёл большую популярность среди коллег и вскоре Страуструп уже не мог лично поддерживать его, отвечая на тысячи вопросов.

К 1983 году в язык были добавлены новые возможности, такие как виртуальные функции, перегрузка функций и операторов, ссылки, константы, пользовательский контроль над управлением свободной памятью, улучшенная проверка типов и новый стиль комментариев (//). Получившийся язык уже перестал быть просто дополненной версией классического C и был переименован из C с классами в «C++». Его первый коммерческий выпуск состоялся в октябре 1985 года.
"""

БЕЗ ОШИБОК

In [4]:
mes, matr, c32, cs=coding(t) 

In [5]:
te=encoding(mes, matr, c32, cs)

контрольная сумма корректна
сообщение доставлено верно


In [6]:
print(t==te)

True


In [7]:
def eq(m1,m2):
    flag=True
    for i in range(len(m1)):
      for j in range (len(m1[0])):
        if m1[i][j]!=m2[i][j]:
          flag=False
    return flag 

Не более одной ошибки


In [8]:
m1, mat1, c1, cs1=coding(t)
m2, mat2, c2, cs2=coding(t)

In [9]:
import random

In [10]:
#внесение ошибок
for i in range(m1.shape[0]):
  j=random.randint(0,123)
  m1[i][j]=1-m1[i][j]

In [11]:
eq(m1,m2)

False

In [12]:
encoding(m1,mat1,c1,cs1)==encoding(m2,mat2,c2,cs2)

контрольная сумма корректна
сообщение доставлено верно
контрольная сумма корректна
сообщение доставлено верно


True

In [13]:
t==encoding(m1,mat1,c1,cs1)

контрольная сумма корректна
сообщение доставлено верно


True

Более одной ошибки 

In [14]:
m3, mat3, c3, cs3=coding(t)

In [15]:
n=0
import random
for i in range(m3.shape[0]):
  j=random.randint(10,56)
  n+=1
  m3[i][j]=1-m3[i][j]
  r=random.choice([True, False])
  k=random.choice(list(range(-10,10)))
  if r:
    m3[i][j+k]=1-m3[i][j+k]
    if (j<25 and j+k>=25) or (j>=25 and j<46 and (j+k<25 or j+k>=46)) or (j>=46 and j+k<46):
      n+=1
#ошибки допущены в (символах):      
print(n/(m3.shape[0]*3))
print(n)

0.36670639650377435
923


In [16]:
dec=encoding(m3,mat3,c3,cs3)

контрольная сумма корректна
в доставленном сообщении имеются ошибки


In [17]:
dec==t

False

In [18]:
err=0
for i in range(len(dec)):
  if dec[i]!=t[i]:
    err+=1
print(err/len(dec))
print(err)

0.8786311181854357
2208


In [19]:
#исправлено ошибок 
print(n-err)

-1285


In [20]:
tr=0
f=0
for i in range(0,len(t)-2,3):
  if dec[i]==t[i] and dec[i+1]==t[i+1] and dec[i+2]==t[i+2]:
    tr+=1
  else:
    f+=1

In [21]:
print(tr,f)

60 778
